In [16]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [17]:
file_path = "sofmattress_train.csv"
df = pd.read_csv(file_path)

In [18]:
df = df.drop_duplicates().reset_index(drop=True)
df["sentence"] = df["sentence"].str.lower().str.strip()

In [19]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])
num_classes = len(label_encoder.classes_)

In [20]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["sentence"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)

In [21]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [22]:
class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [23]:
train_dataset = IntentDataset(train_texts, train_labels, tokenizer)
val_dataset = IntentDataset(val_texts, val_labels, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [24]:
class IntentClassifier(nn.Module):
    def __init__(self, num_classes):
        super(IntentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        return self.fc(x)

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = IntentClassifier(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

In [26]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        correct, total = 0, 0
        for batch in train_loader:
            input_ids, attention_mask, labels = (
                batch["input_ids"].to(device),
                batch["attention_mask"].to(device),
                batch["labels"].to(device)
            )
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            correct += (outputs.argmax(dim=1) == labels).sum().item()
            total += labels.size(0)
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}, Accuracy: {correct/total:.4f}")

In [27]:
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=5)

Epoch 1, Loss: 3.0284271890466865, Accuracy: 0.0541
Epoch 2, Loss: 2.823540564739343, Accuracy: 0.1892
Epoch 3, Loss: 2.5482918493675464, Accuracy: 0.3243
Epoch 4, Loss: 2.241538719697432, Accuracy: 0.5290
Epoch 5, Loss: 1.8898507754007976, Accuracy: 0.6795


In [28]:
torch.save(model.state_dict(), "intent_model.pth")
import pickle
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [29]:
def predict_intent(text, model, tokenizer, label_encoder, max_len=64):
    model.eval()
    encoding = tokenizer(
        text, padding='max_length', truncation=True, max_length=max_len, return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        output = model(input_ids, attention_mask)
        predicted_class = output.argmax(dim=1).item()
    return label_encoder.inverse_transform([predicted_class])[0]

In [30]:
model.load_state_dict(torch.load("intent_model.pth", map_location=device))
model.to(device)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

In [31]:
while True:
    user_input = input("Enter a sentence (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break
    intent = predict_intent(user_input, model, tokenizer, label_encoder)
    print(f"Predicted Intent: {intent}")

Predicted Intent: ORDER_STATUS
Predicted Intent: LEAD_GEN
Predicted Intent: LEAD_GEN
Predicted Intent: DELAY_IN_DELIVERY
Predicted Intent: DELAY_IN_DELIVERY
Predicted Intent: ORDER_STATUS
